# VAE Practical: Inelastic Neutron Scattering

In [CNN_practical.ipynb](../CNN/CNN_practical.ipynb), we have used the inelastic neutron scattering (INS) dataset to practice classification with a convolutional neural network (CNN). In this notebook, we try to make a disentangled variational autoencoder ($\beta$-VAE) to generate new images from the INS dataset, using CNNs for encoding and decoding. 

Compared to a simple VAE, a $\beta$-VAE only introduce one hyperparameter $\beta$ to the loss function to balance the effects of the reconstruction loss and the variational loss. A simple VAE is the special case with $\beta=1$. Learn details of $\beta$-VAE in [VAE_advanced.ipynb](VAE_advanced.ipynb).

In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# check version
print('Using TensorFlow v%s' % tf.__version__)
acc_str = 'accuracy' if tf.__version__[:2] == '2.' else 'acc'

# helpers
import h5py
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. **Even you are not using Google Colab, please make sure you run these two cells.** 

To access the data from Google Colab, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will have no side effects.

In [ ]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop'
colab_data_path = '/content/sciml-workshop-data/'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    with open('../local_data_path.txt', 'r') as f: data_path = f.read().splitlines()[0]

In [ ]:
%%bash -s {google_colab_env} {colab_data_path} {project_id} {bucket_name}

# running locally
if ! $1; then
    echo "Running notebook locally."
    exit
fi

# already mounted
if [ -d $2 ]; then
    echo "Data already mounted."
    exit
fi

# mount the bucket
echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
apt -qq update
apt -qq install gcsfuse
gcloud config set project $3
mkdir $2
gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 $4 $2

---

# The dataset

This section is the same as in [CNN_practical.ipynb](../CNN/CNN_practical.ipynb).

In [ ]:
# define image size
IMG_HEIGHT = 20
IMG_WIDTH = 200
N_CHANNELS = 1
N_CLASSES = 2

# generator
def hdf5_generator(path, buffer_size=32):
    """ Load data INS data from disk
    
    Args:
        path: path of the HDF5 file on disk
        buffer_size: number of images to read from disk
    """
    with h5py.File(path, 'r') as handle:
        n_samples, h, w, c = handle['images'].shape
        for i in range(0, n_samples, buffer_size):
            images = handle['images'][i:i+buffer_size, ..., :1]
            labels = handle['labels'][i:i+buffer_size]
            yield images, labels

# training data
train_dataset = tf.data.Dataset.from_generator(lambda: hdf5_generator(path=data_path + 'ins-data/train.h5'), 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=((None, IMG_HEIGHT, IMG_WIDTH, N_CHANNELS), 
                                                              (None, N_CLASSES,)))

# test data
test_dataset = tf.data.Dataset.from_generator(lambda: hdf5_generator(path=data_path + 'ins-data/test.h5'), 
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=((None, IMG_HEIGHT, IMG_WIDTH, N_CHANNELS), 
                                                             (None, N_CLASSES,)))
# print
print(train_dataset)
print(test_dataset)

In [ ]:
# load the first buffer (with 32 data by default)
images, labels = list(test_dataset.take(1))[0]

# plot some images and labels from it
nplot = 10
fig, axes = plt.subplots(nplot // 2, 2, figsize=(16, nplot / 1.5), dpi=100)
for ax, image, label in zip(axes.flatten(), images, labels):
    ax.matshow(np.squeeze(image))
    ax.set_xlabel('0: Dimer' if label[0] < .5 else '1: Goodenough', c='k')
    ax.set_xticks([])
    ax.set_yticks([])

---

# $\beta$-VAE for  Image Generation

This $\beta$-VAE can be a combination of the CNN architecture in [CNN_practical.ipynb](../CNN/CNN_practical.ipynb) and the $\beta$-VAE implementation in [VAE_advanced.ipynb](VAE_advanced.ipynb).


## 1. Encoder and decoder

First, we need to specify the latent dimension:

In [ ]:
# latent dimension
latent_dim = 32

Now, extend the CNN in [CNN_practical.ipynb](../CNN/CNN_practical.ipynb) to an encoder and a decoder. 

**Suggested Answer for Encoder** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# sampling z with (z_mean, z_log_var)
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# build the encoder
image_input = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, N_CHANNELS))
x = layers.Conv2D(8, kernel_size=(5, 5), activation='relu', padding='same')(image_input)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z_output = Sampling()([z_mean, z_log_var])
encoder_BVAE = keras.Model(image_input, [z_mean, z_log_var, z_output])
encoder_BVAE.summary()
```
    
</p>
</details>


**Suggested Answer for Decoder** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# build the decoder
z_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(800, activation="relu")(z_input)
x = layers.Reshape((2, 25, 16))(x)
x = layers.UpSampling2D(size=(2, 2))(x)
x = layers.Conv2DTranspose(16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D(size=(2, 2))(x)
x = layers.ZeroPadding2D((1, 0))(x)
x = layers.Conv2DTranspose(16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D(size=(2, 2))(x)
x = layers.Conv2DTranspose(8, kernel_size=(5, 5), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
image_output = layers.Conv2DTranspose(1, kernel_size=(3, 3), activation='linear', padding='same')(x)
decoder_BVAE = keras.Model(z_input, image_output)
decoder_BVAE.summary()
```
    
</p>
</details>

## 2. The `BVAE` class

The `BVAE` class can be the same as implemented in [VAE_basics.ipynb](VAE_basics.ipynb) except that we need to pass and use $\beta$. 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# BVAE class
class BVAE(keras.Model):
    # constructor
    ########################################################
    ######## NEW: passing beta as an extra argument ########
    ########################################################
    def __init__(self, encoder, decoder, beta, **kwargs):
        super(BVAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta

    # customise train_step() to implement the loss 
    def train_step(self, x):
        if isinstance(x, tuple):
            x = x[0]
        with tf.GradientTape() as tape:
            # encoding
            z_mean, z_log_var, z = self.encoder(x)
            # decoding
            x_prime = self.decoder(z)
            # reconstruction error by binary crossentropy loss
            reconstruction_loss = tf.reduce_mean(keras.losses.binary_crossentropy(x, x_prime))
            reconstruction_loss *= IMG_HEIGHT * IMG_WIDTH
            # KL divergence
            kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            # loss = reconstruction error + KL divergence
            #######################################
            ######## NEW: scale KL by beta ########
            #######################################
            loss = reconstruction_loss + self.beta * kl_loss
        # apply gradient
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        # return loss for metrics log
        return {"loss": loss,
                "reconstruction_loss": reconstruction_loss,
                "beta_kl_loss": self.beta * kl_loss}
```
    
</p>
</details>

## 3. Build and train the `BVAE` model

Now, build the `BVAE` model and train it with the INS dataset. Let us first use $\beta=5$ and start with 50 epochs. 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# build the BVAE
bvae_model = BVAE(encoder_BVAE, decoder_BVAE, beta=5.)

# compile the BVAE
bvae_model.compile(optimizer='adam')
    
# train the BVAE
training_history_BAVE = bvae_model.fit(train_dataset, epochs=50, batch_size=32)
```
    
</p>
</details>

## 4. Generate images

Finally, we can generate new images using the decoder. After 50 epochs, the generated images resemble the original ones but look pretty vague. We can increase the definition by using more convolutional layers and a larger latent dimension (and thus more epochs) and by tuning the value of $\beta$.



**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# generate images from the latent space
def generate_images_latent(decoder, n_generation, feature_range):
    # randomly sample the latent space
    latent = []
    for dim in range(latent_dim):
        if len(np.array(feature_range).shape) == 1:
            # only one range provided; used it for all dimensions
            latent.append(np.random.uniform(feature_range[0], feature_range[1], 
                                            n_generation))
        else:
            # range provided for each dimension
            latent.append(np.random.uniform(feature_range[dim][0], feature_range[dim][1], 
                                            n_generation))
    latent = np.array(latent).T
    
    # decode images
    decodings = decoder.predict(latent)
    
    # plot generated images
    fig, axes = plt.subplots(n_generation // 2, 2, figsize=(16, n_generation / 2), dpi=100)
    for ax, image in zip(axes.flatten(), decodings):
        ax.matshow(image[:, :, 0])
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()  

# generate random images sampled between [-1, 1]
generate_images_latent(decoder_BVAE, n_generation=30, feature_range=[-1, 1])
```
    
</p>
</details>


---

## Exercises:

1. Tune `latent_dim` and `beta` (and use more epochs) to improve the quality of image generation.
2. Implement a conditional VAE for this INS dataset.